# classifications

with machine learning it never says for certain that the prediction is correct. It only says that this image 
has the highest probability of being correct. 

In [1]:
import torch 
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image
import sklearn
from sklearn.datasets import make_circles
import nbimporter

In [3]:
# importing all pictures
import import_ipynb
import testing


NameError: name 'all_images' is not defined

In [ ]:
# gpu if available
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
img = '/Pytorch/Images/Classification.png'
classification_image = Image(img, width=750, height=750)
classification_image

In the image above
batch_size is equal to none or 32. 
* This is saying how many images the computer should look at, at one time
* **It is noted that batch size is most efficient while using a number devisable by 8**

The number 3 is because we are using 3 layers of classification. 
Looking at the output we can see that we are trying to determine 
weather the image that was provided is sushi meat or a pizza.

The last 2 layers are the size/dimensions of the image provided width by height.  


# Make classification data and get it ready

* The process is the same ML_process that it has always been.
* preparing data step 1

In [ ]:
# making 1000 samples
n_samples = 1000

# create circles
X, y = make_circles(n_samples,# 1
                    noise=0.03, # this is used to create some randomness inside of the code
                    random_state=42) ## this is the same as setting a random seed so that we can have re-producible code. 


In [ ]:
len(X), len(y)

In [ ]:
print(f"fist 5 of x:\n {X[:5]}\n first 5 of y:\n {y[:5]}")

In [ ]:
# making a DataFrame 
import pandas as pd
circles = pd.DataFrame({"X1:": X[:,0],
                        "X2": X[:,1],
                        "label":y})
circles.head(10)

In [ ]:
# looking at it on a graph
plt.scatter(x=X[:,0],
            y=X[:,1],
            c=y,
            cmap=plt.cm.RdYlBu)


# Turning data into tensors.
* Also step 1 in ML process

In [ ]:
# going back to the fundamentals one of the most common errors is that the shapes don't match. 
# So make sure that the input and output match 
X.shape, y.shape

In [ ]:
# view the first example
X_sample = X[0]
y_sample = y[0]

print(f"values for one sample of x: {X_sample} and the same for y: {y_sample}")
print(f"Shapes for one sample of x: {X_sample.shape} and the same for y: {y_sample.shape}")

In [ ]:
# Turning data into tensors
# changing into tensor default datatype
X = torch.from_numpy(X).type(torch.float32)
y = torch.from_numpy(y).type(torch.float32)

X[:5], y[:5] 

# Spiting the data Training and test sets

In [ ]:
# Splitting the data randomly
from sklearn.model_selection import train_test_split

# must be written in this order if using this method to split data. 
X_train, X_test, y_train, y_test = train_test_split(X, # the testing data
                                                    y, # labels
                                                    test_size=0.2, # how big the test split will be (.2 = 20%)
                                                    random_state=42) # set the random seed same as torch.manuel_seed(42)
len(X_train), len(X_test), len(y_train), len(y_test)

# Building a model

Model to classify our blue and red dots

To do 
1. setup device agonistic code so our code will run on an GPU (completed at the top already)
2. Construct a model(by subclassing nn.model)
3. Define a loss function and optimizer
4. Create a training and test loop

In [ ]:
# device agonistic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

# Constructing a model
1. subclass nn.model (almost all pytorch models use this)
2. create 2 nn.Linear() layers that are able to handle the shapes of our data
3. defines a forward() method that outlines the forward pass (or forward compotation) of hte model
4. Insatiate an instance of our model class and send it to the target device

# Model
Note that if you have more than one self.layer the in_features of the layer must be the same as the 
out_features of the previous layer. 

Out features
* This also act like a hidden layer
* the more out_features that a model has the more of a chance the computer has to find patterns in the data,
but there is a upper limit to this (dimensioning return). 


In [ ]:
X_train.shape

In [ ]:
# 1. constructing a model 
class CircleModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        # 2. create 2 nn.Linear layers that can handle the shape of the data
        # We take in 2 in_features because X_train shape is 2 columns
        self.layer_1 = nn.Linear(in_features=2, out_features=5) # takes in 2 features and out put 5
        
        # THE IN FEATURES OF THIS LAYER MUST MATCH THE OUT FEATURES OF THE PREVIOUS LAYER
        self.layer_2 = nn.Linear(in_features=5, out_features=1) 

    # 3. defines a forward() method that outlines the forward pass
    def forward(self, x):
        # Codes backwards but works from the inside out. 
        return self.layer_2(self.layer_1(x)) # x -> layer_1 -> layer_2 -> output
    
# Instantiate an instance of our model class and send it to the target device
model_0 = CircleModelV0().to(device)
model_0

In [ ]:
# checking that the model is on the gpu
next(model_0.parameters()).device

* For a more complex model it is better to use subclassing. (above)
* For a simpler model it's better and easier to use sequential 

In [ ]:
# replicating the model above using nn.Sequential()
model_0 = nn.Sequential(nn.Linear(in_features=2, out_features=5),
                        nn.Linear(in_features=5, out_features=1)).to(device)
model_0

In [ ]:
model_0.state_dict()

In [ ]:
# Make predictions 
with torch.inference_mode():
    untrained_preds = model_0(X_test.to(device))
print(f"Length of predictions: {len(untrained_preds)}, Shape: {untrained_preds.shape}")
print(f"Length of test sample: {len(X_test)}, Shape: {X_test.shape}")
print(f"\n First 10 predictions: \n{untrained_preds[:10]}")
print(f"\n Firs 10 labels: \n {y_test[:10]}")

# setting up the loss function
picking the loss function 
* For regression using MAE or MSE will most likely be the way to go
* For classification using binary cross entropy or categorical cross entropy will be the way to go

Remember loss function measures how wrong th module is

In [ ]:
# some common loss functions are
from IPython.display import Image
loss_functions = Image('Images/common_loss_functions.png', width=750, height=750)
loss_functions